In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
filings_url = 'https://www.sec.gov/cgi-bin/browse-edgar?'
header = {'User-Agent': 'XtraByte Consulting, Inc., bkowalczyk@xtrabyteconsulting.com',
          'Accept': 'application/json',
          'Accept-Encoding': 'gzip, deflate'}
params = {'type' : '4',
          'owner' : 'include',
          'count' : '10',
          'action' : 'getcurrent',
          'output' : 'atom'}

latest_filings = requests.get(filings_url, params=params, headers=header)

In [3]:
if latest_filings.status_code == 200:
    print("Successfully retrieved the latest filings.")

Successfully retrieved the latest filings.


In [4]:
latest_filings

<Response [200]>

In [5]:
import lxml
print(lxml.__version__)


5.3.0


In [7]:
latest_filings_soup = BeautifulSoup(latest_filings.text, 'lxml-xml')
latest_filings_soup.contents


[<feed xmlns="http://www.w3.org/2005/Atom">
 <title>Latest Filings - Fri, 06 Dec 2024 18:10:01 EST</title>
 <link href="/cgi-bin/browse-edgar?action=getcurrent" rel="alternate"/>
 <link href="/cgi-bin/browse-edgar?action=getcurrent" rel="self"/>
 <id>https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent</id>
 <author><name>Webmaster</name><email>webmaster@sec.gov</email></author>
 <updated>2024-12-06T18:10:01-05:00</updated>
 <entry>
 <title>4 - Ju Chi-Hong (0001882831) (Reporting)</title>
 <link href="https://www.sec.gov/Archives/edgar/data/1882831/000128026324000056/0001280263-24-000056-index.htm" rel="alternate" type="text/html"/>
 <summary type="html">
  &lt;b&gt;Filed:&lt;/b&gt; 2024-12-06 &lt;b&gt;AccNo:&lt;/b&gt; 0001280263-24-000056 &lt;b&gt;Size:&lt;/b&gt; 6 KB
 </summary>
 <updated>2024-12-06T18:09:05-05:00</updated>
 <category label="form type" scheme="https://www.sec.gov/" term="4"/>
 <id>urn:tag:sec.gov,2008:accession-number=0001280263-24-000056</id>
 </entry>
 <entry

In [8]:
entries = latest_filings_soup.find_all('entry')
filings = []

for entry in entries:
    filing = {}
    filing['title'] = entry.title.text
    filing['link'] = entry.link['href']
    filing['updated'] = entry.updated.text
    filings.append(filing)
    
filings_df = pd.DataFrame(filings)
filings_df.head()

,title,link,updated
0,4 - Ju Chi-Hong (0001882831) (Reporting),https://www.sec.gov/Archives/edgar/data/188283...,2024-12-06T18:09:05-05:00
1,4 - AMBARELLA INC (0001280263) (Issuer),https://www.sec.gov/Archives/edgar/data/128026...,2024-12-06T18:09:05-05:00
2,4 - Michalek Libor (0001832810) (Reporting),https://www.sec.gov/Archives/edgar/data/183281...,2024-12-06T18:08:11-05:00
3,"4 - Affirm Holdings, Inc. (0001820953) (Issuer)",https://www.sec.gov/Archives/edgar/data/182095...,2024-12-06T18:08:11-05:00
4,"4 - Affirm Holdings, Inc. (0001820953) (Issuer)",https://www.sec.gov/Archives/edgar/data/182095...,2024-12-06T18:08:00-05:00


In [47]:
filings

[{'title': '4 - Ju Chi-Hong (0001882831) (Reporting)',
  'link': 'https://www.sec.gov/Archives/edgar/data/1882831/000128026324000056/0001280263-24-000056-index.htm',
  'updated': '2024-12-06T18:09:05-05:00',
  'file': '/Archives/edgar/data/1280263/000128026324000056/edgardoc.xml'},
 {'title': '4 - AMBARELLA INC (0001280263) (Issuer)',
  'link': 'https://www.sec.gov/Archives/edgar/data/1280263/000128026324000056/0001280263-24-000056-index.htm',
  'updated': '2024-12-06T18:09:05-05:00',
  'file': '/Archives/edgar/data/1280263/000128026324000056/edgardoc.xml'},
 {'title': '4 - Michalek Libor (0001832810) (Reporting)',
  'link': 'https://www.sec.gov/Archives/edgar/data/1832810/000162828024050266/0001628280-24-050266-index.htm',
  'updated': '2024-12-06T18:08:11-05:00',
  'file': '/Archives/edgar/data/1820953/000162828024050266/wk-form4_1733526484.xml'},
 {'title': '4 - Affirm Holdings, Inc. (0001820953) (Issuer)',
  'link': 'https://www.sec.gov/Archives/edgar/data/1820953/00016282802405026

In [46]:
for filing in filings:
    filing_url = filing['link']
    filing_page = requests.get(filing_url, headers=header)
    filing_page_soup = BeautifulSoup(filing_page.text, 'lxml')
    filing_page_soup.title

    
    tables = filing_page_soup.find_all('table')
    for table in tables:
        #if table.caption:
        #    if table.caption.text == 'Document Format Files':
        rows = table.find_all('tr')
        i = 0
        for row in rows:
            cells = row.find_all('td')
            if i == 2:
                if cells:
                    if cells[2].a:
                        filing['file'] = cells[2].a['href']
                        print(filing['file'])
                        # break
            i += 1


/Archives/edgar/data/1280263/000128026324000056/edgardoc.xml
/Archives/edgar/data/1280263/000128026324000056/edgardoc.xml
/Archives/edgar/data/1820953/000162828024050266/wk-form4_1733526484.xml
/Archives/edgar/data/1820953/000162828024050266/wk-form4_1733526484.xml
/Archives/edgar/data/1700539/000162828024050265/wk-form4_1733526474.xml
/Archives/edgar/data/1700539/000162828024050265/wk-form4_1733526474.xml
/Archives/edgar/data/1295402/000095017024134243/ownership.xml
/Archives/edgar/data/1295402/000095017024134243/ownership.xml
/Archives/edgar/data/1058290/000105829024000381/wk-form4_1733526265.xml
/Archives/edgar/data/1058290/000105829024000381/wk-form4_1733526265.xml


In [ ]:
filing_data = []
for filing in filings:
    file_data = {}
    file_url = filing['file']
    file = requests.get('https://www.sec.gov' + file_url, headers=header)
    file_soup = BeautifulSoup(file.text, 'lxml-xml')
    file_data['Symbol'] = file_soup.find('issuerTradingSymbol').text.strip('\n')
    file_data['Issuer CIK'] = file_soup.find('issuerCik').text.strip('\n')
    file_data['Company'] = file_soup.find('issuerName').text.strip('\n')
    file_data['Owner'] = file_soup.find('rptOwnerName').text.strip('\n')
    file_data['Owner CIK'] = file_soup.find('rptOwnerCik').text.strip('\n')
    file_data['IsDirector'] = file_soup.find('isDirector').text.strip('\n')
    file_data['IsOfficer'] = file_soup.find('isOfficer').text.strip('\n')
    file_data['IsTenPercentOwner'] = file_soup.find('isTenPercentOwner').text.strip('\n')
    file_data['IsOther'] = file_soup.find('isOther').text.strip('\n')
    file_data['Officer Title'] = file_soup.find('officerTitle').text.strip('\n')
    file_data['Relationship'] = file_soup.find('rptOwnerRelationship').text.strip('\n')
    transactions = file_soup.find_all('nonDerivativeTransaction')
    file_data['Shares'] = file_soup.find('transactionShares').text.strip('\n')
    file_data['Price'] = file_soup.find('transactionPricePerShare').text.strip('\n')
    file_data['Acquired/Disposed'] = file_soup.find('transactionAcquiredDisposedCode').text.strip('\n')
    file_data['Date'] = file_soup.find('transactionDate').text.strip('\n')
    file_data['Code'] = file_soup.find('transactionCode').text.strip('\n')
    file_data['Equity Swap'] = file_soup.find('equitySwapInvolved').text.strip('\n')
    file_data['Shares Owned'] = file_soup.find('sharesOwnedFollowingTransaction').text.strip('\n')
    filing_data.append(file_data)
    
filing_data_df = pd.DataFrame(filing_data)
filing_data_df

,Symbol,Company,Owner,Shares,Price,Acquired/Disposed,Date,Code,Equity Swap,Shares Owned
0,AMBA,AMBARELLA INC,Ju Chi-Hong,5000,0.0,D,2024-12-04,G,0,164397
1,AMBA,AMBARELLA INC,Ju Chi-Hong,5000,0.0,D,2024-12-04,G,0,164397
2,AFRM,"Affirm Holdings, Inc.",Michalek Libor,93371,8.80,A,2024-12-04,M,0,253972
3,AFRM,"Affirm Holdings, Inc.",Michalek Libor,93371,8.80,A,2024-12-04,M,0,253972
4,AFRM,"Affirm Holdings, Inc.",Liew Jeremy,50000,0,D,2024-12-05,G,0,308713
5,AFRM,"Affirm Holdings, Inc.",Liew Jeremy,50000,0,D,2024-12-05,G,0,308713
6,TEAM,Atlassian Corp,Salem Enrique T,1048,0,A,2024-12-06,A,false,106615
7,TEAM,Atlassian Corp,Salem Enrique T,1048,0,A,2024-12-06,A,false,106615
8,CTSH,COGNIZANT TECHNOLOGY SOLUTIONS CORP,Dalal Jatin P,2688,,A,2024-12-04,M,0,17160
9,CTSH,COGNIZANT TECHNOLOGY SOLUTIONS CORP,Dalal Jatin P,2688,,A,2024-12-04,M,0,17160


In [ ]:
from lxml import etree

for filing in filings:
    file_url = filing['file']
    file = requests.get('https://www.sec.gov' + file_url, headers=header)
    root = etree.fromstring(file.content)
    data = []
    for element in root.iter():
        tag = etree.QName(element).localname
        value = element.text.strip() if element.text else None
        attributes = element.attrib
        data.append(data.append({
            "Tag": tag,
            "Value": value,
            "Attributes": attributes
        }))

In [17]:
all_tables = latest_filings_soup.find_all('table', summary="")
all_tables

[<table bgcolor="white" border="0" cellpadding="0" cellspacing="0" summary="" width="100%">
 <tr>
 <td rowspan="2" width="95"><a href="/index.htm">
 <img alt="U.S. Securities &amp; Exchange Commission" border="0" height="92" src="/images/bannerSeal.gif" width="95"/></a></td>
 <td colspan="2"><img alt="" height="7" src="/images/pixel.gif" width="1"/></td>
 </tr>
 <tr>
 <td width="26"><img alt="SEC Seal" border="0" height="85" src="/images/bannerSealR.gif" width="26"/></td>
 <td valign="top" width="100%">
 <table border="0" cellpadding="0" cellspacing="0" summary="" width="100%">
 <tr>
 <td bgcolor="#324395" valign="top" width="171"><img alt="" border="0" height="30" src="/images/bannerFlag.gif" width="171"/></td>
 <td align="right" bgcolor="#324395" class="gray" nowrap="nowrap" valign="middle" width="95%">
 <a href="/index.htm">Home</a> | <a href="/edgar/searchedgar/webusers.htm">EDGAR Search Home</a> |
  <a href="/cgi-bin/browse-edgar?action=getcurrent">Latest Filings</a> |
  <a href="

In [18]:
filings_table = None
for table in all_tables:
    headers = table.find_all('th')
    if headers:
        header_texts = [header.get_text(strip=True) for header in headers]
        if "Form" in header_texts and "Formats" in header_texts and "Description" in header_texts:
            filings_table = table
            print("Found the filings table.")
            print(filings_table)

Found the filings table.
<table summary="">
<tr bgcolor="#D6D6D6">
<th align="left" nowrap="nowrap" width="4%">Form</th>
<th align="left" nowrap="nowrap" width="4%">Formats</th>
<th align="left" width="40%">Description</th>
<th align="left" nowrap="nowrap" width="4%">Accepted</th>
<th align="left" nowrap="nowrap" width="4%">Filing Date</th>
<th align="left" nowrap="nowrap" width="4%">File/Film No</th>
</tr>
<tr><td>
<td>
<td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>
</td></td></tr>
<tr align="left" nowrap="nowrap" valign="top">
<td nowrap="nowrap">4</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24-028439-index.htm">[html]</a><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24-028439.txt">[text]</a></td>
<td class="small">Statement of changes in beneficial ownership of securi

In [21]:
# Check if we found the filings table
if filings_table:
    # Process the table as before
    filings = []
    for row in filings_table.find_all('tr')[1:]:  # Skip the header row
        cells = row.find_all('td')
        print('Next Row')
        print(cells)
        if len(cells) > 0:
            try:
                # Extract form type
                form_type = cells[0].get_text(strip=True)
                print('Form Type: ' + form_type)

                # Extract description
                description = cells[2].get_text(strip=True)
                print('Description: ' + description)

                # Extract filing date
                filing_date = cells[4].get_text(strip=True)
                print('Filing Date: ' + filing_date)

                # Extract filing links
                links = cells[1].find_all('a')
                form_links = {
                    link.get_text(strip=True): f"https://www.sec.gov{link['href']}" for link in links
                }
                print('Form Links: ' + str(form_links))

                # Append the filing info to the list
                filings.append({
                    "form_type": form_type,
                    "description": description,
                    "filing_date": filing_date,
                    "form_links": form_links,
                })
            except Exception as e:
                print(f"Error parsing row: {e}")

    # Output the extracted filings
    for filing in filings:
        print(filing)
else:
    print("Filings table not found.")

Next Row
[<td>
<td>
<td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>
</td></td>, <td>
<td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>
</td>, <td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>]
Form Type: Lentell Eric (0002016416) (Reporting)
Description: Lentell Eric (0002016416) (Reporting)
Error parsing row: list index out of range
Next Row
[<td nowrap="nowrap">4</td>, <td nowrap="nowrap"><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24-028439-index.htm">[html]</a><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24

In [14]:
entries = latest_filings_soup.find_all('td')
print(entries)


[<td rowspan="2" width="95"><a href="/index.htm">
<img alt="U.S. Securities &amp; Exchange Commission" border="0" height="92" src="/images/bannerSeal.gif" width="95"/></a></td>, <td colspan="2"><img alt="" height="7" src="/images/pixel.gif" width="1"/></td>, <td width="26"><img alt="SEC Seal" border="0" height="85" src="/images/bannerSealR.gif" width="26"/></td>, <td valign="top" width="100%">
<table border="0" cellpadding="0" cellspacing="0" summary="" width="100%">
<tr>
<td bgcolor="#324395" valign="top" width="171"><img alt="" border="0" height="30" src="/images/bannerFlag.gif" width="171"/></td>
<td align="right" bgcolor="#324395" class="gray" nowrap="nowrap" valign="middle" width="95%">
<a href="/index.htm">Home</a> | <a href="/edgar/searchedgar/webusers.htm">EDGAR Search Home</a> |
 <a href="/cgi-bin/browse-edgar?action=getcurrent">Latest Filings</a> |
 <a href="javascript:history.back()">Previous Page</a></td>
<td bgcolor="#324395" width="100%"><img alt="" src="/images/pixel.gif

In [ ]:
filings = []
for entry in entries:
    filing = {}
    filing['title'] = entry.find('title').text
    filing['link'] = entry.find('link')['href']
    filing['updated'] = entry.find('updated').text
    filings.append(filing)

print(filings)


In [ ]:
    
filings_df = pd.DataFrame(filings)
filings_df